In [1]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO
import csv
import os
from copy import deepcopy

from datetime import datetime

In [2]:
folder_name = "VS_02.구매행동_02.선택"

root = "/data/ephemeral/home/datasets/normal/" + folder_name + "/"


header = [
    "Frame",
    "ID",
    "X",
    "Y",
    "Width",
    "Height",
    "Keypoint_0",
    "Keypoint_1",
    "Keypoint_2",
    "Keypoint_3",
    "Keypoint_4",
    "Keypoint_5",
    "Keypoint_6",
    "Keypoint_7",
    "Keypoint_8",
    "Keypoint_9",
    "Keypoint_10",
    "Keypoint_11",
    "Keypoint_12",
    "Keypoint_13",
    "Keypoint_14",
    "Keypoint_15",
    "Keypoint_16",
    "Keypoint_17",
    "Keypoint_18",
    "Keypoint_19",
    "Keypoint_20",
    "Keypoint_21",
    "Keypoint_22",
    "Keypoint_23",
    "Keypoint_24",
    "Keypoint_25",
    "Keypoint_26",
    "Keypoint_27",
    "Keypoint_28",
    "Keypoint_29",
    "Keypoint_30",
    "Keypoint_31",
    "Keypoint_32",
    "Keypoint_33",
]

In [3]:
with open(f"{folder_name}.csv", "w") as c_file:
    writer = csv.writer(c_file, delimiter=",")

    writer.writerow(header)

In [4]:
file_list = os.listdir(root)
print(f"==>> file_list: {file_list}")
file_list.sort()
print(f"==>> file_list: {file_list}")

==>> file_list: ['C_2_2_60_BU_SMA_09-16_15-59-50_CB_RGB_DF1_M4_M4.mp4', 'C_2_2_60_BU_SYB_10-15_09-59-03_CA_RGB_DF1_M4_M4.mp4', 'C_2_2_59_BU_SYB_10-15_17-34-57_CB_RGB_DF1_F3_F3.mp4', 'C_2_2_65_BU_SYA_09-16_14-08-57_CD_RGB_DF1_F4_F4.mp4', 'C_2_2_61_BU_SYA_09-16_14-04-18_CC_RGB_DF1_M4_M4.mp4', 'C_2_2_59_BU_DYB_10-20_14-02-41_CC_RGB_DF1_M4_M4.mp4', 'C_2_2_61_BU_SYB_10-15_10-00-40_CA_RGB_DF1_M4_M4.mp4', 'C_2_2_60_BU_SYB_10-15_09-59-03_CD_RGB_DF1_M4_M4.mp4', 'C_2_2_59_BU_SYA_09-15_10-50-47_CD_RGB_DF1_F3_F3.mp4', 'C_2_2_63_BU_SMB_10-15_11-00-36_CD_RGB_DF1_M4_M4.mp4', 'C_2_2_58_BU_DYB_10-20_14-00-58_CC_RGB_DF1_M4_M4.mp4', 'C_2_2_61_BU_SYA_09-16_14-04-18_CD_RGB_DF1_M4_M4.mp4', 'C_2_2_59_BU_SMA_09-16_10-07-19_CB_RGB_DF1_F3_F3.mp4', 'C_2_2_57_BU_SYB_10-15_17-33-01_CB_RGB_DF1_F3_F3.mp4', 'C_2_2_61_BU_SYB_10-15_10-00-40_CD_RGB_DF1_M4_M4.mp4', 'C_2_2_63_BU_SMB_10-15_11-00-36_CC_RGB_DF1_M4_M4.mp4', 'C_2_2_58_BU_DYB_10-20_14-00-58_CD_RGB_DF1_M4_M4.mp4', 'C_2_2_65_BU_SYB_10-15_10-03-46_CB_RGB_DF1_F4_F4

In [5]:
# Load the YOLOv8 model
model = YOLO("yolov8n-pose.pt")

# Define the standard frame size (change these values as needed)
standard_width = 640
standard_height = 480

In [6]:
# # test
# file_list = file_list[:5]
# print(f"==>> file_list: {file_list}")

==>> file_list: ['C_1_1_93_BU_DYB_10-19_11-51-57_CC_DF1_F4_F4.mp4', 'C_1_1_93_BU_DYB_10-19_11-51-57_CE_DF1_F4_F4.mp4', 'C_1_1_88_BU_DYA_07-18_16-17-30_d_DF1_F2_M2.mp4', 'C_1_1_88_BU_DYB_10-19_10-56-16_CD_DF1_F4_F4.mp4', 'C_1_1_92_BU_DYA_07-18_16-42-53_a_DF1_F2_F2.mp4']


In [6]:
# Loop through the video frames
frame_count = 0

id_count = 0

for file_name in file_list:
    path = root + file_name

    time_start = datetime.now()

    print(f"{file_name} feature extracting starts")

    cap = cv2.VideoCapture(path)

    # Store the track history
    track_history = defaultdict(lambda: [])

    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()

        frame_count += 1  # Increment frame count

        if success:
            frame = cv2.resize(frame, (standard_width, standard_height))

            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, persist=True, verbose=False)

            if results[0].boxes is not None:  # Check if there are results and boxes
                # Get the boxes
                # boxes = results[0].boxes.xywh.cpu()

                if results[0].boxes.id is not None:
                    # If 'int' attribute exists (there are detections), get the track IDs
                    track_ids = results[0].boxes.id.int().cpu().tolist()

                    for i, box in zip(range(0, len(track_ids)), results[0].boxes.xywhn.cpu()):
                        keypoints = results[0].keypoints.xyn[i].cpu().numpy().flatten().tolist()
                        box_list = box.numpy().flatten().tolist()
                        if type(box_list) == 'float' or type(keypoints) == 'float':
                            print(f"==>> box_list: {box_list}")
                            print(f"==>> keypoints: {keypoints}")
                        box_and_keypoints = box_list + keypoints
                        track_history[track_ids[i]].append([[frame_count], deepcopy(box_and_keypoints)])
        else:
            # Break the loop if the end of the video is reached
            break

    with open(f"{folder_name}.csv", "a") as c_file:
        writer = csv.writer(c_file, delimiter=",")
        for key in track_history.keys():
            for f_count, b_and_k in track_history[key]:
                row = f_count + [id_count+key] + b_and_k

                writer.writerow(row)

    id_count = id_count + len(track_history.keys())

    cap.release()

    time_end = datetime.now()
    total_time = time_end - time_start
    total_time = str(total_time).split(".")[0]

    print(f"{file_name} feature extracting ended. Elapsed time: {total_time}")

# cv2.destroyAllWindows()

C_2_2_57_BU_DYB_10-20_13-59-11_CA_RGB_DF1_M4_M4.mp4 feature extracting starts
C_2_2_57_BU_DYB_10-20_13-59-11_CA_RGB_DF1_M4_M4.mp4 feature extracting ended. Elapsed time: 0:00:22
C_2_2_57_BU_DYB_10-20_13-59-11_CB_RGB_DF1_M4_M4.mp4 feature extracting starts
C_2_2_57_BU_DYB_10-20_13-59-11_CB_RGB_DF1_M4_M4.mp4 feature extracting ended. Elapsed time: 0:00:19
C_2_2_57_BU_DYB_10-20_13-59-11_CC_RGB_DF1_M4_M4.mp4 feature extracting starts
C_2_2_57_BU_DYB_10-20_13-59-11_CC_RGB_DF1_M4_M4.mp4 feature extracting ended. Elapsed time: 0:00:19
C_2_2_57_BU_DYB_10-20_13-59-11_CD_RGB_DF1_M4_M4.mp4 feature extracting starts
C_2_2_57_BU_DYB_10-20_13-59-11_CD_RGB_DF1_M4_M4.mp4 feature extracting ended. Elapsed time: 0:00:19
C_2_2_57_BU_SMA_09-16_10-05-32_CC_RGB_DF1_F3_F3.mp4 feature extracting starts
C_2_2_57_BU_SMA_09-16_10-05-32_CC_RGB_DF1_F3_F3.mp4 feature extracting ended. Elapsed time: 0:00:19
C_2_2_57_BU_SMA_09-16_10-05-33_CA_RGB_DF1_F3_F3.mp4 feature extracting starts
C_2_2_57_BU_SMA_09-16_10-05-33_C

error: OpenCV(4.9.0) /io/opencv/modules/highgui/src/window.cpp:1266: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
